# My XGBoost implementation

This time, plan is a little diffrent. I will exceptionally not implement the code from scratch, because I would probably need C++ efficiency.
Main goal of these mini-projects was to have handy and not complicated "notebook" written in Python to refresh my knowledge later in my journey. XGBoost is more computationally difficult, so it would disrupt the initial assumption for this project.

Of course, I could implement it on smaller dataset, but does it make sense when XGBoost was made for really large datasets?

So, this time I will do only two steps.

    1. Notes for XGBoost
    2. Implementation from libary

In [20]:
print("Let's get to work!")

Let's get to work!


## 1. Notes
    A. Main idea
    B. How it works
    C. Math behind the algorithm
    D. When to use it?

### A. Main idea

**XGBoost** is an algorithm that builds many decision trees, where <u> each new tree corrects the errors of the previous ones.</u> <br>
It is based on Gradient Boosting, but introduces optimizations that make it faster and more effective on large and complex datasets. <br>

Key improvments are:

    * Regularization to reduce overfitting -> adds λ penalties to prevent overfitting and improve generalization.
  
    * Efficient handling of missing values -> automatically learns the best path for missing data, no need for preprocessing.
  
    * Parallelization for speed  -> computes tree splits in parallel, making training much faster on large datasets.
  
    * Sparsity awareness and memory optimization -> optimized for sparse data and memory usage with efficient data structures.


### B. How it works

1) Start with an initial prediction (e.g., 0.5 for log-loss classification, or the mean of y for regression).
2) Calculating <u>Residuals</u> for every example (Residual = prediction - real_data).
3) Building first leaf that contains all training samples.
4) For the first leaf, we calculate <u>Similarity Score</u>
   
   ![](Sim_score.png)

   in easier language...


   ![](sim_score_2.png) 

5) After that, we split the residuals into two groups in many possible ways. Suppose we have m residuals: in the first scenario, one residual goes to the left leaf and the remaining m-1 to the right; in the next scenario, two residuals go left and m-2 go right; and so on.
6) For each scenario we calculate <u>Gain</u>
   
   ![](gain.png) 

   ![](gain2.png)

   We chceck gain in every scenario, and pick the biggest one. The winner becomes a new node.

7) This pattern continues, making new nodes until it reaches the max depth.

8) However, we pick γ - parameter for prunning tree. Now we check if gain - γ is negative or not. If it is, we prune a node, and check if prevous node is also negative.

9) Calculating output values
    
    ![](output_val.png)


10) Update predictions: new prediction = old prediction + 𝜂⋅ (leaf weight), where 𝜂 is the learning rate.

    ![](n_pred.png)

quick summary at 23:57 minute -> https://www.youtube.com/watch?v=OtD8wVaFm6E

### C. Algorithm in Math

### D. When to use it?

* ✅ When you have large and complex datasets

* ✅ When data contains missing values

* ✅ When you need high accuracy 

* ✅ Works with diffrent types of data

## 2. Implementation from library

In [21]:
# Importing libraries

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import xgboost as xgb
import numpy as np
from datetime import datetime



In [27]:
# Importing dataset 

df = pd.read_csv(r"C:\Users\wikto\OneDrive\Dokumenty\all-datasets\Lung Cancer.csv")
df.head()

df = df.drop('id', axis=1)

df.head()


# Prepering Data

# 1. Getting days passed since diagnosis
df["diagnosis_date"] = pd.to_datetime(df["diagnosis_date"])
df["end_treatment_date"] = pd.to_datetime(df["end_treatment_date"])

treatment_time = (df["diagnosis_date"] - df["end_treatment_date"]).abs()

df["treatment_time"] = treatment_time
df.head()

# 2. Fixing category type of data

df["gender"] = df["gender"].astype('category')
df["country"] = df["country"].astype('category')
df["cancer_stage"] = df["cancer_stage"].astype('category')
df["smoking_status"] = df["smoking_status"].astype('category')
df["treatment_type"] = df["treatment_type"].astype('category')

df.head()

,age,gender,country,diagnosis_date,cancer_stage,family_history,smoking_status,bmi,cholesterol_level,hypertension,asthma,cirrhosis,other_cancer,treatment_type,end_treatment_date,survived,treatment_time
0,64.0,Male,Sweden,2016-04-05,Stage I,Yes,Passive Smoker,29.4,199,0,0,1,0,Chemotherapy,2017-09-10,0,523 days
1,50.0,Female,Netherlands,2023-04-20,Stage III,Yes,Passive Smoker,41.2,280,1,1,0,0,Surgery,2024-06-17,1,424 days
2,65.0,Female,Hungary,2023-04-05,Stage III,Yes,Former Smoker,44.0,268,1,1,0,0,Combined,2024-04-09,0,370 days
3,51.0,Female,Belgium,2016-02-05,Stage I,No,Passive Smoker,43.0,241,1,1,0,0,Chemotherapy,2017-04-23,0,443 days
4,37.0,Male,Luxembourg,2023-11-29,Stage I,No,Passive Smoker,19.7,178,0,0,0,0,Combined,2025-01-08,0,406 days
